In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2023-04-21 19:05:32.704866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 19:05:33.466077: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.__version__

'2.12.0'

### Pre-process Image Data

In [3]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [4]:
training_set = train_datagen.flow_from_directory(
    '../data/cnn_dataset/training_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 8000 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
    '../data/cnn_dataset/test_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.


### Building the CNN

In [6]:
cnn = tf.keras.models.Sequential()

2023-04-21 19:05:34.509047: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-21 19:05:34.540479: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


#### Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

#### Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#### Adding Second Convolution Layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#### Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

#### Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#### Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Training the CNN

#### Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#### Training the CNN on the Training Set + Evaluation on Test Set

In [15]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


2023-04-21 19:05:34.858838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - ETA: 0s - loss: 0.6703 - accuracy: 0.5863

2023-04-21 19:05:59.109567: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - 28s 109ms/step - loss: 0.6703 - accuracy: 0.5863 - val_loss: 0.6290 - val_accuracy: 0.6495
Epoch 2/25
250/250 [==============================] - 29s 114ms/step - loss: 0.6136 - accuracy: 0.6619 - val_loss: 0.5669 - val_accuracy: 0.7095
Epoch 3/25
250/250 [==============================] - 28s 112ms/step - loss: 0.5690 - accuracy: 0.7031 - val_loss: 0.5416 - val_accuracy: 0.7365
Epoch 4/25
250/250 [==============================] - 29s 115ms/step - loss: 0.5479 - accuracy: 0.7170 - val_loss: 0.5425 - val_accuracy: 0.7245
Epoch 5/25
250/250 [==============================] - 28s 114ms/step - loss: 0.5184 - accuracy: 0.7414 - val_loss: 0.5473 - val_accuracy: 0.7210
Epoch 6/25
250/250 [==============================] - 28s 112ms/step - loss: 0.4963 - accuracy: 0.7599 - val_loss: 0.5071 - val_accuracy: 0.7525
Epoch 7/25
250/250 [==============================] - 28s 113ms/step - loss: 0.4786 - accuracy: 0.7650 - val_loss: 0.4911 - val_accuracy: 0.7

In [19]:
import numpy as np
import keras.utils as image

test_image = image.load_img('../data/cnn_dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = cnn.predict(test_image)

# training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

print(prediction)

1/1 [==============================] - 0s 14ms/step
dog
